# Rankingを生成
- 前の試合にhome及びawayのチームが何位であったかを算出
- 従って、第1節は欠損となる点に注意
- idとteamをキーとして結合すれば説明変数の作成可

### おまじない

In [ ]:
import pandas as pd
import numpy as np

### データの読み込み

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train_add = pd.read_csv("train_add.csv")
test_add = pd.read_csv("2014_add.csv")
condition = pd.read_csv("condition.csv")
cond_add = pd.read_csv("condition_add.csv")

### データの結合

In [ ]:
tmp = pd.concat([train,train_add,test,test_add],sort=False).reset_index(drop=True).sort_values(by="id")
cond = pd.concat([condition,cond_add],sort=False).reset_index(drop=True).sort_values(by="id")

In [ ]:
tmp["match_num"] = tmp["match"].apply(lambda x : int(x[x.find("第")+1:x.find("節")]))
cols = ["id","match_num","home","away","year","gameday","stage"]
rank = pd.merge(tmp[cols],cond[["home_score","away_score","id"]],on="id")
rank

### 各試合の結果から勝ち点を計算し、scoreとして算出

In [ ]:
rank["h_score"] = rank.apply(lambda x : 3*(x["home_score"]>x["away_score"])+1*(x["home_score"]==x["away_score"]),axis=1)
rank["a_score"] = rank.apply(lambda x : 3*(x["home_score"]<x["away_score"])+1*(x["home_score"]==x["away_score"]),axis=1)
tmp1 = rank.iloc[:,~rank.columns.str.contains("away")].rename(columns={"home":"team","h_score":"score"})
tmp2 = rank.iloc[:,~rank.columns.str.contains("home")].rename(columns={"away":"team","a_score":"score"})
tmp = pd.concat([tmp1,tmp2],sort=False).drop(columns=["a_score","h_score","away_score","home_score"])
tmp = tmp.sort_values(by="id").reset_index(drop=True)
tmp.head(17)

### 勝ち点を累積させ、各節における順位を算出

In [ ]:
tmp["scores"] = tmp.groupby(by=["year","team"]).cumsum()["score"]
tmp["rank"] = tmp.groupby(by=["year","stage","match_num"])["scores"].rank(method="min",ascending=False)
tmp

### 次の試合の時に、現在何位であるかとするため、前の試合における順位をshiftさせる
- その為、第一節はまだ試合を実施していないため欠損とする

In [ ]:
tmp.sort_values(by=["team","year","match_num"],inplace=True)
tmp["rank_"] = tmp["rank"].shift(1)
tmp["rank_"] = tmp.apply(lambda x : np.nan if x["match_num"]==1 else x["rank_"], axis=1)
tmp.to_csv("ranking.csv",index=None,encoding="utf8")

### train_all及びtest_allに順位データを結合

In [ ]:
train = pd.read_csv("train_all.csv")
test = pd.read_csv("test_all.csv")

In [ ]:
train_home_rank = pd.merge(train[["id","home"]],tmp[["id","team","rank_"]],left_on=["id","home"],right_on=["id","team"])["rank_"]
train_away_rank = pd.merge(train[["id","away"]],tmp[["id","team","rank_"]],left_on=["id","away"],right_on=["id","team"])["rank_"]
train["home_rank"] = train_home_rank
train["away_rank"] = train_away_rank
train

In [ ]:
test_home_rank = pd.merge(test[["id","home"]],tmp[["id","team","rank_"]],left_on=["id","home"],right_on=["id","team"])["rank_"]
test_away_rank = pd.merge(test[["id","away"]],tmp[["id","team","rank_"]],left_on=["id","away"],right_on=["id","team"])["rank_"]
test["home_rank"] = test_home_rank
test["away_rank"] = test_away_rank
test

In [ ]:
train.to_csv("train_all2.csv",encoding="utf8")
test.to_csv("test_all2.csv",encoding="utf8")